In [ ]:
! pip install langchain-community

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

Document Loaders

In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()

[Document(metadata={'source': './index.md'}, page_content='\nLista de Tareas de DOMINGO:\n\n\n1. Ir al gimnasio\n2. Terminar la serie\n3. Poner una lavadora\n4. Hacer meal prepping\n            ')]

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='./example_data/mlb_teams_2012.csv')
data = loader.load()

In [ ]:
! pip install pypdf

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("example_data/meeting_data.pdf")
pages = loader.load_and_split()

In [8]:
pages[0]

Document(metadata={'source': 'example_data/meeting_data.pdf', 'page': 0}, page_content='Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.')

Text Splitter

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)


chunks_pages = text_splitter.split_documents(pages)

Text embedding models

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [12]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [13]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.005377273540943861,
 -0.0006527779041789472,
 0.038980286568403244,
 -0.002967397216707468,
 -0.008834563195705414]

Vector stores

In [ ]:
%pip install -qU langchain-qdrant

In [43]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

raw_documents = PyPDFLoader("example_data/meeting_data.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [53]:
qdrant = QdrantVectorStore.from_documents(
    documents=documents,
    embedding= embeddings,
    location=":memory:",  
    collection_name="my_documents",
)

query = "¿Cuándo fue la última reunión?"
found_docs = qdrant.similarity_search(query)

In [45]:
found_docs

[Document(metadata={'source': 'example_data/meeting_data.pdf', 'page': 0, '_id': '8b8d3323f7b4457886ddc60bd632380c', '_collection_name': 'my_documents'}, page_content='Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.')]

In [46]:
query = "Cuándo fue la última reunión?"
found_docs = await qdrant.amax_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

1. Reunión de planificación - 15 de agosto de 2024
Asistentes:
- Juan Pérez
- María González
- Luis Fernández
- Ana Martínez
Agenda:
1. Revisión de los objetivos del trimestre pasado.
2. Planificación de las próximas actividades.
3. Asignación de responsabilidades.
4. Preguntas y respuestas.
Conclusiones:
- Se logró el 85% de los objetivos del trimestre pasado.
- Se identificaron áreas de mejora en la comunicación interna.
- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la
capacitación del personal.
La próxima reunión está programada para el 20 de septiembre de 2024. 



Retrievers

In [61]:
retriever = qdrant.as_retriever(search_type="mmr") # Maximum Marginal Relevance (MMR)
docs = retriever.invoke("¿Cuándo fue la última reunión?")

In [63]:
docs

[Document(metadata={'source': 'example_data/meeting_data.pdf', 'page': 0, '_id': '5982ab570976435db0d3e8ad96fed2a3', '_collection_name': 'my_documents'}, page_content='Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.')]

Utiliza built-in chains para almacenar, recuperar y generar

In [64]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""
question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [78]:

from langchain_core.messages import HumanMessage

input = "¿Cuándo fue la última reunión?"

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content=f"{input}")
        ],
    }
)


'La última reunión fue el 15 de agosto de 2024.'

In [79]:
from langchain import hub
from langchain.chains import create_retrieval_chain

retrieval_qa_chat_prompt = hub.pull("lime-97/rag_chain_demo_prompt")

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

response = retrieval_chain.invoke({"input":"¿quiénes intervienen en la reunión?"})
response['answer']

'En la reunión intervienen Juan Pérez, María González, Luis Fernández y Ana Martínez.'